# Pipeline ELT – Transform Data

Nama     : [Nama Kamu]
Peran    : ELT
Topik    : Analitik Transportasi dan Mobilitas Perkotaan

Deskripsi:
Notebook ini digunakan untuk melakukan proses transformasi data
pada pipeline ELT menggunakan SQL. Transformasi mencakup data cuaca
dan data operasional Transjakarta yang sebelumnya telah dimuat
ke dalam data warehouse dalam bentuk tabel raw.


# MOUNT DRIVE & PATH

In [8]:
from google.colab import drive
drive.mount('/content/drive')

PROJECT_ROOT = "/content/drive/MyDrive/! semester 7/bigdata_final_project"
WAREHOUSE_PATH = f"{PROJECT_ROOT}/warehouse"
DB_PATH = f"{WAREHOUSE_PATH}/bigdata_warehouse.db"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
import os
print(os.path.exists(DB_PATH))


True


# IMPORT & CONNECT SQLITE

In [10]:
import sqlite3
import pandas as pd
from datetime import datetime

conn = sqlite3.connect(DB_PATH)


# TRANSFORM WEATHER

## CEK INPUT WEATHER RAW

In [11]:
pd.read_sql(
    "SELECT * FROM raw_weather_hourly LIMIT 5;",
    conn
)


,time,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
0,2023-04-01 00:00:00,26.8,22.6,78.0,0.0,None,250.0,5.4,None,1010.8,None,3.0
1,2023-04-01 01:00:00,26.1,22.8,82.0,0.0,None,255.0,18.4,None,1010.7,None,3.0
2,2023-04-01 02:00:00,27.0,23.2,80.0,0.0,None,254.0,18.0,None,1010.9,None,3.0
3,2023-04-01 03:00:00,26.2,23.5,85.0,0.0,None,270.0,9.4,None,1011.9,None,3.0
4,2023-04-01 04:00:00,29.0,24.1,75.0,0.0,None,267.0,18.0,None,1010.2,None,3.0


## TRANSFORM WEATHER (SQL)

Tambah kolom jam (hour)

Handle hujan (is_rainy)

Tandai jam sibuk (is_rush_hour)

Tetap simpan kolom numerik utama

Semua dilakukan di SQL (ELT)

In [12]:
conn.executescript("""
DROP TABLE IF EXISTS elt_clean_weather_hourly;

CREATE TABLE elt_clean_weather_hourly AS
SELECT
    time,
    CAST(strftime('%H', time) AS INTEGER) AS hour,
    temp,
    rhum,
    COALESCE(prcp, 0) AS prcp,
    wspd,
    pres,
    coco,
    CASE
        WHEN COALESCE(prcp, 0) > 0 THEN 1
        ELSE 0
    END AS is_rainy,
    CASE
        WHEN CAST(strftime('%H', time) AS INTEGER) BETWEEN 6 AND 9
          OR CAST(strftime('%H', time) AS INTEGER) BETWEEN 16 AND 19
        THEN 1
        ELSE 0
    END AS is_rush_hour
FROM raw_weather_hourly;
""")


## VERIFIKASI HASIL WEATHER CLEAN

In [13]:
pd.read_sql(
    "SELECT * FROM elt_clean_weather_hourly LIMIT 10;",
    conn
)


,time,hour,temp,rhum,prcp,wspd,pres,coco,is_rainy,is_rush_hour
0,2023-04-01 00:00:00,0,26.8,78.0,0.0,5.4,1010.8,3.0,0,0
1,2023-04-01 01:00:00,1,26.1,82.0,0.0,18.4,1010.7,3.0,0,0
2,2023-04-01 02:00:00,2,27.0,80.0,0.0,18.0,1010.9,3.0,0,0
3,2023-04-01 03:00:00,3,26.2,85.0,0.0,9.4,1011.9,3.0,0,0
4,2023-04-01 04:00:00,4,29.0,75.0,0.0,18.0,1010.2,3.0,0,0
5,2023-04-01 05:00:00,5,28.2,77.0,0.3,21.6,1009.4,8.0,1,0
6,2023-04-01 06:00:00,6,29.6,70.0,0.2,9.4,1009.0,7.0,1,1
7,2023-04-01 07:00:00,7,27.9,76.0,2.0,17.3,1007.9,9.0,1,1
8,2023-04-01 08:00:00,8,27.7,80.0,0.5,13.0,1007.4,8.0,1,1
9,2023-04-01 09:00:00,9,29.8,62.0,0.0,9.4,1007.7,3.0,0,1


CEK STRUKTUR TABEL

In [14]:
pd.read_sql(
    "PRAGMA table_info(elt_clean_weather_hourly);",
    conn
)


,cid,name,type,notnull,dflt_value,pk
0,0,time,NUM,0,None,0
1,1,hour,INT,0,None,0
2,2,temp,REAL,0,None,0
3,3,rhum,REAL,0,None,0
4,4,prcp,,0,None,0
5,5,wspd,REAL,0,None,0
6,6,pres,REAL,0,None,0
7,7,coco,REAL,0,None,0
8,8,is_rainy,,0,None,0
9,9,is_rush_hour,,0,None,0


## LOGGING TRANSFORM WEATHER

In [15]:
row_count = pd.read_sql(
    "SELECT COUNT(*) AS cnt FROM elt_clean_weather_hourly;",
    conn
)["cnt"][0]

print("=== LOG ELT – TRANSFORM WEATHER ===")
print("Input Table  : raw_weather_hourly")
print("Output Table : elt_clean_weather_hourly")
print("Jumlah Baris :", row_count)
print("Waktu Proses :", datetime.now())


=== LOG ELT – TRANSFORM WEATHER ===
Input Table  : raw_weather_hourly
Output Table : elt_clean_weather_hourly
Jumlah Baris : 744
Waktu Proses : 2026-01-06 13:35:15.402626


# TRANSFORM TRANSJAKARTA

## CEK INPUT RAW TRANSJAKARTA

In [16]:
pd.read_sql(
    "SELECT * FROM raw_transjakarta LIMIT 5;",
    conn
)


,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,...,tapInStopsLon,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount
0,VRPJ892P3M98RA,3561407960318444,dki,"Dr. Janet Nashiruddin, M.Ak",M,2010,4,Pulo Gadung 2 - Tosari,1.0,P00167,...,106.89165,12,2023-04-03 06:53:02,P00127,Layur,-6.193539,106.89909,13.0,2023-04-03 07:13:28,3500.0
1,ZWCH834I6M26HS,347728053419394,emoney,Balamantri Rahayu,M,2002,JAK.28,Kp. Rambutan - Taman Wiladatika,1.0,B04633P,...,106.86298,27,2023-04-03 05:59:19,B00865P,Jln. Ar Ridho,-6.308148,106.86935,30.0,2023-04-03 06:57:06,0.0
2,YRLD835V6L82GO,377105453850671,emoney,Dian Mustofa,F,1993,B13,Bekasi Barat - Blok M,1.0,B02192P,...,106.99215,6,2023-04-03 05:13:24,B00108P,Bandar Djakarta Bekasi,-6.227085,106.99683,9.0,2023-04-03 06:01:23,20000.0
3,ZZBX143N6N83HQ,4486493302356581,dki,"Cut Janet Suryatmi, M.Ak",M,1980,8K,Batusari - Grogol,1.0,B03637P,...,106.78213,16,2023-04-03 05:20:24,None,Yayasan Alkahfi Jakbar,-6.198896,106.76889,26.0,2023-04-03 06:01:25,3500.0
4,EWEG491A2W45DR,30139379978125,bni,"dr. Mulyanto Pudjiastuti, M.Ak",F,1997,None,None,0.0,None,...,106.83514,2,2023-04-03 06:00:54,B02755P,Perpustakaan Nasional,-6.180673,106.82643,5.0,2023-04-03 06:47:32,3500.0


In [17]:
# cek struktur kolom

pd.read_sql(
    "PRAGMA table_info(raw_transjakarta);",
    conn
)


,cid,name,type,notnull,dflt_value,pk
0,0,transID,TEXT,0,None,0
1,1,payCardID,INTEGER,0,None,0
2,2,payCardBank,TEXT,0,None,0
3,3,payCardName,TEXT,0,None,0
4,4,payCardSex,TEXT,0,None,0
5,5,payCardBirthDate,INTEGER,0,None,0
6,6,corridorID,TEXT,0,None,0
7,7,corridorName,TEXT,0,None,0
8,8,direction,REAL,0,None,0
9,9,tapInStops,TEXT,0,None,0


## TRANSFORM TRANSJAKARTA (SQL)

In [18]:
conn.executescript("""
DROP TABLE IF EXISTS elt_clean_transjakarta;

CREATE TABLE elt_clean_transjakarta AS
SELECT
    *,
    CAST(strftime('%H', tapInTime) AS INTEGER) AS hour
FROM raw_transjakarta;
""")


## VERIFIKASI HASIL TRANSJAKARTA CLEAN

In [19]:
pd.read_sql(
    "SELECT * FROM elt_clean_transjakarta LIMIT 10;",
    conn
)


,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,...,stopStartSeq,tapInTime,tapOutStops,tapOutStopsName,tapOutStopsLat,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,hour
0,VRPJ892P3M98RA,3561407960318444,dki,"Dr. Janet Nashiruddin, M.Ak",M,2010,4,Pulo Gadung 2 - Tosari,1.0,P00167,...,12,2023-04-03 06:53:02,P00127,Layur,-6.193539,106.89909,13.0,2023-04-03 07:13:28,3500.0,6
1,ZWCH834I6M26HS,347728053419394,emoney,Balamantri Rahayu,M,2002,JAK.28,Kp. Rambutan - Taman Wiladatika,1.0,B04633P,...,27,2023-04-03 05:59:19,B00865P,Jln. Ar Ridho,-6.308148,106.86935,30.0,2023-04-03 06:57:06,0.0,5
2,YRLD835V6L82GO,377105453850671,emoney,Dian Mustofa,F,1993,B13,Bekasi Barat - Blok M,1.0,B02192P,...,6,2023-04-03 05:13:24,B00108P,Bandar Djakarta Bekasi,-6.227085,106.99683,9.0,2023-04-03 06:01:23,20000.0,5
3,ZZBX143N6N83HQ,4486493302356581,dki,"Cut Janet Suryatmi, M.Ak",M,1980,8K,Batusari - Grogol,1.0,B03637P,...,16,2023-04-03 05:20:24,None,Yayasan Alkahfi Jakbar,-6.198896,106.76889,26.0,2023-04-03 06:01:25,3500.0,5
4,EWEG491A2W45DR,30139379978125,bni,"dr. Mulyanto Pudjiastuti, M.Ak",F,1997,None,None,0.0,None,...,2,2023-04-03 06:00:54,B02755P,Perpustakaan Nasional,-6.180673,106.82643,5.0,2023-04-03 06:47:32,3500.0,6
5,KTGG277R2I90ZB,30394281967910,bni,"Dasa Prakasa, S.I.Kom",F,1993,JAK.117,Tanjung Priok - Tanah Merdeka,1.0,B04272P,...,17,2023-04-03 05:06:00,B04818P,Simpang Gotong Royong 2,-6.125482,106.89331,21.0,2023-04-03 05:41:04,0.0,5
6,VSJU956Z4T24LI,4974829151599657,dki,Elvina Hasanah,M,1966,9F,Rusun Tambora - Pluit,0.0,P00090,...,5,2023-04-03 06:21:07,P00170,Penjaringan,-6.126306,106.79203,7.0,2023-04-03 06:52:58,0.0,6
7,HBMD368H3K77KW,3532177104809428,dki,"Perkasa Prakasa, M.Pd",F,2011,JAK.31,Blok M - Andara,0.0,B00042P,...,55,2023-04-03 06:34:03,B01147P,Jln. H. Terin Pangkalan Jati,-6.327961,106.80054,56.0,2023-04-03 07:32:53,0.0,6
8,SQHG432Q6Z10BR,3537522260639116,dki,Ayu Wahyudin,F,2009,JAK.37,Cililitan - Condet via Kayu Manis,1.0,B05218P,...,10,2023-04-03 05:48:17,B00405P,Gg. Ani Raya Condet,-6.266674,106.86054,37.0,2023-04-03 06:13:38,0.0,5
9,EHWL093H6L20UL,4677151756111287186,brizzi,"H. Xanana Santoso, S.Farm",M,2010,JAK.15,Bulak Turi - Tanjung Priok,0.0,B01327P,...,46,2023-04-03 05:09:13,B05728P,Term. Tj. Priok 1,-6.109800,106.88118,52.0,2023-04-03 06:07:24,0.0,5


## CEK STRUKTUR TABEL

In [20]:
pd.read_sql(
    "PRAGMA table_info(elt_clean_transjakarta);",
    conn
)


,cid,name,type,notnull,dflt_value,pk
0,0,transID,TEXT,0,None,0
1,1,payCardID,INT,0,None,0
2,2,payCardBank,TEXT,0,None,0
3,3,payCardName,TEXT,0,None,0
4,4,payCardSex,TEXT,0,None,0
5,5,payCardBirthDate,INT,0,None,0
6,6,corridorID,TEXT,0,None,0
7,7,corridorName,TEXT,0,None,0
8,8,direction,REAL,0,None,0
9,9,tapInStops,TEXT,0,None,0


## LOGGING

In [21]:
row_count = pd.read_sql(
    "SELECT COUNT(*) AS cnt FROM elt_clean_transjakarta;",
    conn
)["cnt"][0]

print("=== LOG ELT – TRANSFORM TRANSJAKARTA ===")
print("Input Table  : raw_transjakarta")
print("Output Table : elt_clean_transjakarta")
print("Jumlah Baris :", row_count)
print("Waktu Proses :", datetime.now())


=== LOG ELT – TRANSFORM TRANSJAKARTA ===
Input Table  : raw_transjakarta
Output Table : elt_clean_transjakarta
Jumlah Baris : 189500
Waktu Proses : 2026-01-06 13:35:36.254173


# Fact table row-level enriched

In [22]:
# join + agregasi

# conn.executescript("""
# DROP TABLE IF EXISTS elt_fact_transport_weather;

# CREATE TABLE elt_fact_transport_weather AS
# SELECT
#     DATE(t.tapInTime) AS tanggal,
#     t.hour,
#     w.is_rainy,
#     w.is_rush_hour,
#     COUNT(*) AS total_transaksi,
#     AVG(w.temp) AS avg_temp,
#     SUM(w.prcp) AS total_hujan
# FROM elt_clean_transjakarta t
# JOIN elt_clean_weather_hourly w
#     ON DATE(t.tapInTime) = DATE(w.time)
#    AND t.hour = w.hour
# GROUP BY
#     DATE(t.tapInTime),
#     t.hour,
#     w.is_rainy,
#     w.is_rush_hour;
# """)

# join + enrichment

conn.executescript("""
DROP TABLE IF EXISTS elt_fact_transport_weather;

CREATE TABLE elt_fact_transport_weather AS
SELECT
    t.*,              -- 1 baris = 1 transaksi Transjakarta
    w.temp,
    w.rhum,
    w.prcp,
    w.is_rainy,
    w.is_rush_hour
FROM elt_clean_transjakarta t
JOIN elt_clean_weather_hourly w
    ON DATE(t.tapInTime) = DATE(w.time)
   AND t.hour = w.hour;
""")



## VERIFIKASI FACT TABLE

In [23]:
pd.read_sql(
    "SELECT * FROM elt_fact_transport_weather LIMIT 10;",
    conn
)


,transID,payCardID,payCardBank,payCardName,payCardSex,payCardBirthDate,corridorID,corridorName,direction,tapInStops,...,tapOutStopsLon,stopEndSeq,tapOutTime,payAmount,hour,temp,rhum,prcp,is_rainy,is_rush_hour
0,VRPJ892P3M98RA,3561407960318444,dki,"Dr. Janet Nashiruddin, M.Ak",M,2010,4,Pulo Gadung 2 - Tosari,1.0,P00167,...,106.89909,13.0,2023-04-03 07:13:28,3500.0,6,30.8,65.0,0.0,0,1
1,ZWCH834I6M26HS,347728053419394,emoney,Balamantri Rahayu,M,2002,JAK.28,Kp. Rambutan - Taman Wiladatika,1.0,B04633P,...,106.86935,30.0,2023-04-03 06:57:06,0.0,5,30.8,65.0,0.0,0,0
2,YRLD835V6L82GO,377105453850671,emoney,Dian Mustofa,F,1993,B13,Bekasi Barat - Blok M,1.0,B02192P,...,106.99683,9.0,2023-04-03 06:01:23,20000.0,5,30.8,65.0,0.0,0,0
3,ZZBX143N6N83HQ,4486493302356581,dki,"Cut Janet Suryatmi, M.Ak",M,1980,8K,Batusari - Grogol,1.0,B03637P,...,106.76889,26.0,2023-04-03 06:01:25,3500.0,5,30.8,65.0,0.0,0,0
4,EWEG491A2W45DR,30139379978125,bni,"dr. Mulyanto Pudjiastuti, M.Ak",F,1997,None,None,0.0,None,...,106.82643,5.0,2023-04-03 06:47:32,3500.0,6,30.8,65.0,0.0,0,1
5,KTGG277R2I90ZB,30394281967910,bni,"Dasa Prakasa, S.I.Kom",F,1993,JAK.117,Tanjung Priok - Tanah Merdeka,1.0,B04272P,...,106.89331,21.0,2023-04-03 05:41:04,0.0,5,30.8,65.0,0.0,0,0
6,VSJU956Z4T24LI,4974829151599657,dki,Elvina Hasanah,M,1966,9F,Rusun Tambora - Pluit,0.0,P00090,...,106.79203,7.0,2023-04-03 06:52:58,0.0,6,30.8,65.0,0.0,0,1
7,HBMD368H3K77KW,3532177104809428,dki,"Perkasa Prakasa, M.Pd",F,2011,JAK.31,Blok M - Andara,0.0,B00042P,...,106.80054,56.0,2023-04-03 07:32:53,0.0,6,30.8,65.0,0.0,0,1
8,SQHG432Q6Z10BR,3537522260639116,dki,Ayu Wahyudin,F,2009,JAK.37,Cililitan - Condet via Kayu Manis,1.0,B05218P,...,106.86054,37.0,2023-04-03 06:13:38,0.0,5,30.8,65.0,0.0,0,0
9,EHWL093H6L20UL,4677151756111287186,brizzi,"H. Xanana Santoso, S.Farm",M,2010,JAK.15,Bulak Turi - Tanjung Priok,0.0,B01327P,...,106.88118,52.0,2023-04-03 06:07:24,0.0,5,30.8,65.0,0.0,0,0


## CEK STRUKTUR TABEL

In [24]:
pd.read_sql(
    "PRAGMA table_info(elt_fact_transport_weather);",
    conn
)


,cid,name,type,notnull,dflt_value,pk
0,0,transID,TEXT,0,None,0
1,1,payCardID,INT,0,None,0
2,2,payCardBank,TEXT,0,None,0
3,3,payCardName,TEXT,0,None,0
4,4,payCardSex,TEXT,0,None,0
5,5,payCardBirthDate,INT,0,None,0
6,6,corridorID,TEXT,0,None,0
7,7,corridorName,TEXT,0,None,0
8,8,direction,REAL,0,None,0
9,9,tapInStops,TEXT,0,None,0


## LOGGING FACT TABLE

In [25]:
row_count = pd.read_sql(
    "SELECT COUNT(*) AS cnt FROM elt_fact_transport_weather;",
    conn
)["cnt"][0]

print("=== LOG ELT – FACT TABLE ===")
print("Fact Table   : elt_fact_transport_weather")
print("Jumlah Baris:", row_count)
print("Waktu Proses:", datetime.now())


=== LOG ELT – FACT TABLE ===
Fact Table   : elt_fact_transport_weather
Jumlah Baris: 189500
Waktu Proses: 2026-01-06 13:35:53.167953


cek doang

In [26]:
pd.read_sql(
    """
    SELECT COUNT(*) AS total_transaksi_keseluruhan
    FROM elt_fact_transport_weather;
    """,
    conn
)


,total_transaksi_keseluruhan
0,189500


In [27]:
# cek row

pd.read_sql(
    "SELECT COUNT(*) AS jumlah_row FROM elt_fact_transport_weather;",
    conn
)



,jumlah_row
0,189500


In [34]:
pd.read_sql(
    """
    SELECT
        hour,
        COUNT(*) AS transaksi_hujan
    FROM elt_fact_transport_weather
    WHERE is_rainy = 1
    GROUP BY hour
    ORDER BY hour;
    """,
    conn
)


,hour,transaksi_hujan
0,5,4292
1,6,12021
2,7,9078
3,8,11434
4,9,7165
5,10,1005
6,11,552
7,12,248
8,13,344
9,14,123


In [35]:
pd.read_sql(
    """
    SELECT
        DATE(tapInTime) AS tanggal,
        COUNT(*) AS transaksi_hujan
    FROM elt_fact_transport_weather
    WHERE hour = 17
      AND is_rainy = 1
    GROUP BY DATE(tapInTime)
    ORDER BY transaksi_hujan DESC;
    """,
    conn
)


,tanggal,transaksi_hujan
0,2023-04-30,114
